In [2]:
from collections import defaultdict
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
import pyfiglet

from refactorings_detection import *
from util import *
from study_notebook_helper import *


def get_containing_class_of(repo: LocalRepo, path: str) -> str:
    return repo.get_tree().find_node(path).get_containing_class_node().get_path()


ignored_types = SerializedWrap(set(), "../refactorings/ignored_refactoring_types.pickle")


@interact_manual.options(manual_name="Go!")(info=[
    "jfree/jfreechart:v1.5.3::v1.5.0"
])
def main(info):
    repo_name, old_version = info.split("::")
    # 1 get list of refactorings for each old class
    #   a refactoring contains a type and a set of locations within this and other old classes
    # 2 cached, interactively ask the user whether this class has been "refactored to fix its modular structure / architecture"
    #   showing old and new code of the class, and the list of refactorings (grouped by other old location), links to GitHub old and new version of the other locations
    #   While doing so, allow users to filter more refactoring types (also persisted)
    repo = LocalRepo(repo_name)
    repo.get_tree(old_version)
    repo.get_tree()
    old = repo.get_commit(old_version).hexsha
    new = repo.get_head_commit().hexsha

    RefactoringType = Tuple[str, str] # type_name, desc
    results: Dict[str, Dict[str, List[RefactoringType]]] = defaultdict(lambda: defaultdict(lambda: []))  # old class, old other class, ref
    for commit in get_raw_refactorings_per_commit(repo, old, new):
        for ref in commit["refactorings"]:
            if ref["type"] not in ignored_types.data:
                for left_side_path in set(refactoring_process_path(repo, ref["leftSideLocations"])):
                    for right_side_path in set(refactoring_process_path(repo, ref["rightSideLocations"])):
                        results[get_containing_class_of(repo, left_side_path)][left_side_path].append((ref["type"], ref["description"]))
    print(len(results))
    all_types = set(type_name for class_info in results.values() for refs in class_info.values() for type_name, desc in refs)
    
    @interact_manual.options(manual_name="Ignore type!")(ignore=[None] + list(all_types))
    def ignore_type(ignore):
        if ignore is None:
            return
        print(f"Ignoring {ignore}!")
        ignored_types.data.add(ignore)
        ignored_types.save()
        print(pyfiglet.figlet_format("Reloading..."))
        main.widget.manual_button.click()
    
    @interact_manual.options(manual_name="Unignore type!")(unignore=[None] + list(ignored_types.data))
    def ignore_type(unignore):
        if unignore is None:
            return
        print(f"Ignoring {unignore}!")
        ignored_types.data.remove(unignore)
        ignored_types.save()
        print(pyfiglet.figlet_format("Reloading..."))
        main.widget.manual_button.click()
    
    def show_refactoring(class_path: str):

        print("\n" * 5)
        print_html("<h1>Has this class been refactored to fix its modular structure / architecture?</h1>")
        print("\n" * 5)
        
        file_path = class_path.split(".java")[0] + ".java"
        old_code = repo.get_tree(old_version).find_node(class_path).get_comment_and_own_text_formatted(repo.get_file(file_path, old_version))
        new_code = repo.get_tree().find_node(class_path).get_comment_and_own_text_formatted(repo.get_file(file_path))

        print_html(f"""{PRISM_HIGHLIGHTING}{OWN_STYLE}<br>
            You will be shown multiple methods from the <a href="{repo.url()}" target="_blank" style="color: blue">{repo.name}</a> repository.
            Take a close look at the two presented snippets of code.<br>
            Consider everything, not only the statements, but also variable naming, comments,
            location of these methods within the project, (common) editing history, and relations with other code.<br>
            Click on the links to view the presented code snippets on GitHub, where you can find further context information if needed.<br>
            <br>
            Then, please select which of the given categories best describes this pair of methods.<br>
            Even if this is not clear, please pick the one category that you think fits the most to the method pair.
            <br><br><br><br>
            <center>
                Location of class within the project tree:<br>
                <br>
                {make_path_html(repo, "", class_path.split("/"))}
            </center>
            <br><br>
            <table border="1" style="width: calc(0.95 * 100vw - 150px)">
              <tr>
                <td style="font-family: monospace"><center>{path_html(repo, class_path, old_version)}</center></td>
                <td style="font-family: monospace"><center>{path_html(repo, class_path)}</center></td>
              </tr>
              <tr>
                <td style="max-width: 50%"><pre><code class="language-java match-braces rainbow-braces">{format_code(old_code)}</code></pre></td>
                <td style="max-width: 50%"><pre><code class="language-java match-braces rainbow-braces">{format_code(new_code)}</code></pre></td>
              </tr>
            </table>
        """)
        print(results[class_path])
        for other, refs in results[class_path].items():
            print(pyfiglet.figlet_format(other))
            for ref in refs:
                print(ref)
    show_refactoring(list(results.keys())[0])


interactive(children=(Dropdown(description='info', options=('jfree/jfreechart:v1.5.3::v1.5.0',), value='jfree/…